# Geospatial Analysis Problem Set

<span style="color:red">0 / 0 points</span>.

In this problem set, you will continue to work with geospatial California data. After every step in the problem set, create a new cell, code or Markdown as appropriate, for the answer. 

# Programming
1. Make a map of California where each county is colored by its land area; this value is in a shapefile used in the Geospatial chapter.  Remove the x and y axis, do not display a legend, and save the file as 'CA_county_area.jpg' with a resolution of 450 dots per inch.  

In [ ]:
# import the necessary packages
import geopandas as gpd
import matplotlib.pyplot as plt # Necessary for saving

# The shapefile
ca_counties = gpd.read_file('../../Data/California/shapefiles/CA_Counties/CA_Counties_TIGER2016.shp')

# Finding the column name for land area
ca_counties.head()

# The plot
ca_counties.plot(column='ALAND').set_axis_off()
plt.savefig('CA_county_area.jpg', dpi=450, format='jpg')

2. Make the same map but add a legend with the label 'Land area'.  Save it using the same name as in step 1.

In [ ]:
ca_counties.plot(column='ALAND', legend=True, legend_kwds={'label': 'Land area'}).set_axis_off()
plt.savefig('CA_county_area.jpg', dpi=450, format='jpg')

3. Make a new variable that is the percent of each county whose area is water; call it `water_perc`.  What is the mean percent of countries that is water?  Provide the answer using a print statement.

In [ ]:
ca_counties['water_perc'] = ca_counties['AWATER']/(ca_counties['AWATER'] + ca_counties['ALAND'])

ca_counties['water_perc'].describe()  # Inspecting to confirm it works

mean = ca_counties['water_perc'].describe()[1]

print('The mean percent of area in California counties that is water is ' + str(mean*100) + '.')

4. Make a map of California where each county is colored by this new variable.  Display a legend but do not display x and y axes or save.

In [ ]:
ca_counties.plot(column='water_perc', legend=True, legend_kwds={'label': 'Percent Area as Water'}).set_axis_off()

5. Make a map of California where each county is colored by the number of transit stops.  Display a legend and give it a title.  Do not display the axes.  Save the map as 'CA_transitStops_byCounty.jpg' with a 300 dots per inch resolution.

In [ ]:
# Load, inspect data
transit = gpd.read_file('../../Data/California/transit/CA_Transit_Stops.csv')

# Make the geometry column, assuming X is longitude and Y is latitude
transit['geometry'] = gpd.points_from_xy(transit['X'], transit['Y'])

# Assume the projection system is EPSG 3857, planar. 
# planar means meters from the Prime Meridian and equator respectively.
transit.crs = 'epsg:3857'

# Convert to degrees, the longitude and latitude that is most common.
transit = transit.to_crs('epsg:4326')

# Make CA counties data the same coordinate system
ca_counties4326 = ca_counties.to_crs('epsg:4326')

# Make a new column of the county geometry.  The spatial join only keeps the left dataframe's geometry.
ca_counties4326['geometry_later'] = ca_counties4326['geometry']  

# Map stops to county with a spatial join
transit2 = transit.sjoin(ca_counties4326, how='left', predicate='within')

# The below lines are to figure out which column name to use for aggregation, they are not necessary for a correct answer.
import pandas as pd
pd.options.display.max_columns = None

transit2.head()  # Will use COUNTYFP

# Make dataframe to plot
# You can count using any column.  'first' takes the first item per COUNTYFP, giving county name.
transit2_agg = transit2.dissolve(by='COUNTYFP', aggfunc={'stop_id':'count', 'NAMELSAD':'first', 'geometry_later':'first'}, as_index=False)
transit2_agg['geometry'] = transit2_agg['geometry_later']

# Plot
transit2_agg.plot(column='stop_id', legend=True, legend_kwds={'label': 'Transit Stops'}).set_axis_off()
plt.savefig('CA_transitStops_byCounty.jpg', dpi=300, format='jpg')

6. Use the California overcrowding data to make a map of the percent of households overcrowded by county from 2011-2015 regardless of race.  Display a legend and give it a title.  Do not display the axes.  

In [106]:
crowding = gpd.read_file('../../Data/California/housingCrowding/hci_housing_crowding.csv')

crowding = crowding[crowding['geotype']=='CO']  # For county
crowding = crowding[crowding['reportyear']=='2011-2015'] 
crowding = crowding[crowding['race_eth_code']=='9']  # 9 is Total, all races.  In the dataset it is a string variable.
# crowding = crowding[crowding['race_eth_name']=='Total'] # This line also selects for all races

## Merge in geometry for plotting
# Load the shapefile
counties = gpd.read_file('../../Data/California/shapefiles/CA_Counties/CA_Counties_TIGER2016.shp')

# Merge
crowding_shp = crowding.merge(counties, left_on='geoname', right_on='NAME')

crowding_shp['numerator'] = pd.to_numeric(crowding_shp['numerator'])
crowding_shp['denominator'] = pd.to_numeric(crowding_shp['denominator'])

crowding_shp['perc_overcrowded'] = crowding_shp['numerator']/crowding_shp['denominator']

# Get correct geometry name
crowding_shp['geometry'] = crowding_shp['geometry_y']
crowding_shp = crowding_shp.set_geometry('geometry') # If you do not set a geometry, it will not work.  Notice the original dataframe is not a geodataframe.

crowding_shp.plot(column='perc_overcrowded', legend=True, legend_kwds={'label': 'Percent Households Overcrowded'}).set_axis_off()
              


7. Make a map of California cities in Imperial, Kern, Los Angeles, Orange, Riverside, San Bernardino, San Diego, San Luis Obispo, Santa Barbara, and Ventura counties that shows the number of transit stops per city.  This problem is quite long, so here are some hints:

- You need three datasets.
- You should get an error about 'index_right' and 'index_left'.  When that error comes, drop the problematic column(s).
- Before plotting you will need to aggregate stops to the city.  This process will take a very long time, so to save time use `pandas` to take a random subset of 5000 rows chosen with the random_state seed of 1039.  

In [5]:
# You do not need to load the data again but I like to for clarity's sake.
transit = gpd.read_file('../../Data/California/transit/CA_Transit_Stops.csv')
ca_places = gpd.read_file('../../Data/California/shapefiles/ca-places-boundaries/CA_Places_TIGER2016.shp')
ca_counties = gpd.read_file('../../Data/California/shapefiles/CA_Counties/CA_Counties_TIGER2016.shp')

# Keep only the required countries
these = ['Imperial', 'Kern', 'Los Angeles', 'Orange', 'Riverside', 'San Bernardino', 'San Diego', 'San Luis Obispo', 'Santa Barbara', 'Ventura']
ca_counties = ca_counties[ca_counties['NAME'].isin(these)]

# Make the geometry column, assuming X is longitude and Y is latitude
transit['geometry'] = gpd.points_from_xy(transit['X'], transit['Y'])

# Coordinate systems
transit.crs = 'epsg:3857'
transit = transit.to_crs('epsg:4326')
ca_counties = ca_counties.to_crs('epsg:4326')

# Merge, inner keeps only those in the counties
transit = transit.sjoin(ca_counties, how='inner', predicate='within')

## Prepare cities data
# Subset to only cities
ca_cities = ca_places[ca_places['NAMELSAD'].str.contains('city')]
ca_cities = ca_cities.to_crs('epsg:4326')
ca_cities['geometry_later'] = ca_cities['geometry']

# Merge
transit = transit.drop(columns=['index_right'])  # Needed to prevent an error
transit2 = transit.sjoin(ca_cities, how='inner', predicate='within') # inner to keep only those cities in the 
transit2['geometry'] = transit2['geometry_later']  # Makes the geometry the city outlines

# Randomly sample before aggregating.  Otherwise, aggregating will take a very long time.
transit2 = transit2.sample(n=5000, random_state=1039)

# Aggregate to city
transit2_city = transit2.dissolve(by='NAME_right', aggfunc={'stop_id':'count', 'geometry_later':'first', 'NAMELSAD_left': 'first'}, as_index=False)

# Plot
transit2_city.plot(column='stop_id', legend=True, legend_kwds={'label': 'Number of Transit Stops'}).set_axis_off()

8. It is a bit difficult to see city variation when looking at so many counties at once.  For this problem, plot the number of transit stops only in Los Angeles County.

In [ ]:
la_county = transit2_city[transit2_city['NAMELSAD_left']=='Los Angeles County']

la_county.plot(column='stop_id', legend=True, legend_kwds={'label': 'Number of Transit Stops'}).set_axis_off()

NameError: name 'transit2_city' is not defined

# Interpretation
8. According to the map you made of transit stops, Los Angeles County has significantly more transit stops than the other counties, and most of the Southern California counties appear to have as many transit stops as the California counties situated around San Francisco Bay.  This pattern goes against conventional wisdom which says that Southern California is more car centric than the San Francisco Bay area.  What explains why the map shows a different pattern?  If the pattern should match conventional wisdom, what would you change in the plot?

The map shows a different pattern because Southern California counties have many more people than the San Francisco Bay area ones.  Since they have more people, they should have more transit stops even if a greater proportion of people use cars in Southern California than in the San Francisco Bay counties.  To correct for the population imbalance, a more appropriate measure would be transit stops per capita, which requires a county population variable.  Once transit stops per capita is mapped, the pattern should match conventional wisdom.

9. According to the overcrowding map, which county is the most overcrowded?  Where are other overcrowded counties?  What part of the state contains the least overcrowding?

The most overcrowded county is Monterey.  The overcrowded counties are concentrated near the major metropolitan areas.  For example, Los Angeles County appears to have the second highest rate of overcrowding.  The least crowded countries are in the Central and North East, essentially the eastern side of the Sierra Nevada mountain range.

# Debugging

10. Below is code that is supposed to plot the percent overcrowded by county.  It does not work.  Why?

In [ ]:
crowding.plot(coolumn='perc_overcrowded', legend=True, legend_kwds={'label': 'Percent Households Overcrowded'}).set_axis_off()

`coolumn` is misspelled, it should be `column`.

11.  Below is code that should read in the places shapefile.  If you execute it you will find it does not.  Why?

In [ ]:
ca_places = gpd.read_file('CA_Places_TIGER2016.shp')

The path is not relative to this notebook and there is no file with that name in the directory where this notebook is.  A relative path is needed.